#IST664/CIS668 - Homework 3 (assigned in Week 7)#

Note to students: You may freely use code from the labs for this class without the need for attributions. Also note that this HW template has cell tags that are used by the grading program. Please retain these cell tags. Feel free to add new code blocks as needed: these do not need tags.

Originality assertion: All of the text and comments in this file are my original work (except for template items written by the instructor). All of the code in this file is my work, except where I give credit to another source. By adding my name below, I affirm this originality assertion.

*** My name: _________ ***

Sometimes, it can be beneficial to discuss the challenges in the homework with another student. This is fine as long as you DO NOT SHARE CODE with the other person. If you consulted with anyone about this homework, list their names here:

*** My collaborators: _________ ***


**Task 1: Read Sentence Pairs from Github**

It is a weird acronym, but the "Sentences Involving Compositional Knowledge" (SICK) dataset includes a large number of sentence pairs with various levels of similarity. I've stored a copy in my GitHub account and you should be able to download it directly to your virtual machine and into a Pandas dataframe. Here's a link to a web page with more information about the data:
https://marcobaroni.org/composes/sick.html 

When you review the data, you will notice a field where the options are ENTAILMENT, CONTRADICTION, or NEUTRAL. Textual entailment is when one sentence logically follows based on the meaning of another sentence. You can read more about textual entailment here: https://en.wikipedia.org/wiki/Textual_entailment

In [ ]:
import pandas as pd

sick_data = pd.read_csv("https://raw.githubusercontent.com/jmstanto/ist664/main/SICK_train.txt" , sep='\t', on_bad_lines='skip', index_col=None) 

# HW3T1A
# Add code to display the first few rows of data
#



In [ ]:
# HW3T1B
# Add code that shows the number of variables and the number of rows in the dataset.
# Add some comments describing what each of the columns contains. 
#


**Task 2: Add a Similarity Score to Each Row**

Use a pre-trained sentence embedding model to generate a similarity score for each sentence pair. To get started, here's some code from Lab 7:

In [ ]:
# HW3T2A
# The first thing we will need is the library for loading sentence transformers
# This generates a lot of output, but should run pretty fast.
!pip install sentence-transformers

In [ ]:
# HW3T2B
# Now load a pre-trained sentence transformer. There are hundreds to choose from.
# This downloads a lot of data to your virtual machine and takes half a minute or so.
from sentence_transformers import SentenceTransformer

# Here's a sentence transformer model that encodes a d=384 vector. See:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# HW3T2C
# Here's an example of how we can put the sentence transformer to use 
# by encoding the two sentences from the first row and computing a 
# cosine similarity between them.
from sentence_transformers.util import cos_sim

show_row = 2

print(sick_data["sentence_A"][show_row])
print(sick_data["sentence_B"][show_row])
print(sick_data["entailment_judgment"][show_row])
a = model.encode([sick_data["sentence_A"][show_row]])
b = model.encode([sick_data["sentence_B"][show_row]])

cos_sim(a, b).tolist()[0][0] # The notation on the end extracts a scalar from the tensor object

In [ ]:
# HW3T2D
# Add code to produce a similarity score for each sentence pair and insert the
# list into the Pandas dataframe as a new column. You can use the "insert" method
# on a Pandas df to add a column. Choose a sensible label for your new variable as
# you will need it later for the regression analysis.
#
# Note that if you process the whole dataset, it will take a couple of minutes
# to encode all 4500 of the sentences. You can subset the data down to 1000
# or 1500 rows if you prefer to shorten the run time.
#



In [ ]:
# HW3T2E
# Review a few rows of the modified Pandas df with the new column
#


In [ ]:
# HW3T2F
# Run and interpret a simple correlation between your new similarity score
# and the original relatedness_score column. The corr() method from Pandas
# (which is a method for columns) can do this. 
#


Add your interpretation of the correlation by replacing this text. Would you consider this value to be small? Large? Is it what you expected? Why?

**Task 3: Create Dummy Codes for Entailment**

In this task after this one, you will produce a regression analysis that predicts the original relatedness_score from your newly added similarity measure along with two dummy codes created from the entailment_judgment.

With linear regression we can use as predictors any combination of metric variables and binary codes. With a label like entailment_judgment, with three or more categories, we need to create dummy codes to represent the categories. To avoid collinearity, there should always be one less dummy code than the number of categories.

In [ ]:
# HW3T3A
# Show the three types of entailment_judgment using set().


In [ ]:
# HW3T3B
contra = [ int(ej == "CONTRADICTION") for ej in sick_data["entailment_judgment"]]
entail = [ int(ej == "ENTAILMENT") for ej in sick_data["entailment_judgment"]]

In [ ]:
# HW3T3C
# Insert the two new dummy codes into your pandas df
#


In [ ]:
# HW3T3D
# Review the newly modified df
#


**Task 4: See if the root verbs match**

In HW2 you conducted some information extraction from sentences including locating the root verb. If the root verbs match on two sentences, that may indicate something about how related the sentences are. For this task you will create a list of dummy codes that indicate whether the two sentences in each pair use the same root verb.

In [ ]:
# HW3T4A
import spacy # SpaCy will help us find roots
nlp = spacy.load("en_core_web_sm")

In [ ]:
# HW3T4B
# Here's some demo code that shows how to fetch the roots
show_row = 5

print(sick_data["sentence_A"][show_row])
print(sick_data["sentence_B"][show_row])

doc1 = nlp(sick_data["sentence_A"][show_row])
doc2 = nlp(sick_data["sentence_B"][show_row])

list(doc1.sents)[0].root, list(doc2.sents)[0].root

In [ ]:
# HW3T4C
# Here's a test that compares the two strings
list(doc1.sents)[0].root.text == list(doc2.sents)[0].root.text

In [ ]:
# HW3T4D
# Write a loop that will perform the test on each pair of sentences and save the result in a list
#


In [ ]:
# HW3T4E
# Insert the new dummy code into your pandas df.
# Use the column name, "root_match", as shown in the example
#


In [ ]:
# HW3T4F
# Display the pandas df
#


**Task 5: Run Regression Analysis to Predict relatedness_score**

Next, your job is to conduct a regression analysis where you predict relatedness_score from your newly added similarity measure along with two dummy codes created from the entailment_judgment. The sklearn library has a linear regression model, but statsmodels.api produces nicer output.

In [ ]:
# HW3T5A
# The sm.OLS() method does regression and once you have fitted the model
# the summary() method produces nice output with all the values needed to 
# interpret regression results.
#
import statsmodels.api as sm
import statsmodels.formula.api as smf

# You can specify your regression model using a formula language, like this:
# results = smf.ols('relatedness_score ~ similarity + dummy_entail + dummy_contra + root_match', data=sick_data).fit()
# Of course, you may need to adjust the X-variable names depending upon the
# labels you used when you inserted them into the database. 


In [ ]:
# HW3T5B
# Use the summary() method on your results object to show the regression output.

HW3T5C

Replace this text with an explanation of the regression output. Make sure to report on the values of the coefficients for the predictors (and the intercept). Also report and interpret the R-squared value. Is this a good predictive model? 

**Task 6: Find the sentence with the largest error of prediction and the smallest error of prediction.**



In [ ]:
# HW3T6A
# Use the predict() method on the results object to produce predicted scores 
# for each row in your data frame. For the regression equation to work, you must
# supply all three of the X variables for each case. Pandas makes it 
# easy to create a Pandas subset of columns, like this:
#
# predictors = sick_data[['similarity', 'dummy_entail', 'dummy_contra', 'root_match']]


In [ ]:
# HW3T6B
import matplotlib.pyplot as plt

#
# Show a histogram of predicted values from the previous block. Comment on
# why the range of the predicted values does or does not make sense.
#

In [ ]:
# HW3T6C
# Compute a squared error value for each row of your data frame. This is easy:
# subtract the predicted value from the actual value (i.e., relatedness_score) 
# and square the result. You can use a for loop or a list comprehension. 

# Hint 1:
# Python's built-in zip() function allows you to bundle two vectors of data
# together into a list of tuples.
# Hint 2: 
# When extracting data from a Pandas column, sometimes you must use the 
# tolist() method to get the data into a structure that can be used for
# functions like zip().



In [ ]:
# HW3T6D
import numpy as np
# Use np.argmin() to find the index of the row in the Pandas dataset with the 
# smallest squared error of prediction


In [ ]:
# HW3T6E
# Use np.argmax() to find the index of the row with the largest squared error


In [ ]:
# HW3T6F
# Display the Pandas data from the row with the worst prediction. Note that
# the iloc() method allows you to select a particular row.


In [ ]:
# HW3T6G
# Display the Pandas data from the row with the worst prediction


HW3T6H

Replace this text with an explanation of why the argmin() case was "easy" to predict and the argmax() case was "hard" to predict. This explanation should consider the semantic content of the sentences in reference to the relatedness_score developed by the experts.

**Task 7: Write your own two contradictory sentences and predict the relatedness score**

In [ ]:
# HW3T7A
# Replace these empty strings with your own sentences
test_sentenceA = ""
test_sentenceB = ""

In [ ]:
# HW3T7B
# Encode the sentences into vectors as was demonstrated for task 2.


In [ ]:
# HW3T7C
# Compute the cosine similarity between the two vectors.

In [ ]:
# HW3T7D
# You may have used the formula interface for fitting your regression model
# with smf.ols(). If that's true, the easiest way to get a prediction for a
# novel case is to build a one-row Pandas dataframe and enter the X variables
# you need to make the prediction. The next two lines of commented code
# provide an exmaple of this technique: 
# temp_df = pd.DataFrame(columns=['similarity', 'dummy_entail', 'dummy_contra', 'root_match'])
# temp_df.loc[0] = [0.80, 0, 0] # You can include whatever X data you want in place of these three values
#


In [ ]:
# HW3T7E
# Use the predict() method on your regression output object to make one 
# prediction of relatedness_score using the one-row Pandas dataframe from
# the previous step.
#


HW3T7F

Replace this text with an interpretation of the output score from the prediction based on your two sentences.

HWCC

Don't forget to write a concluding comment describing your findings. Provide an overview of your findings from the tasks you accomplished above. How well were you able to predict the relatedness score from the other variables? Did entailment or contradiction work as predictors and what does each coefficient say about our ability to predict relatedness scores? Would it be helpful to know something about entailment if we are trying to understand paraphrases? Why do we care about entailment in the context of an application such as chat?